## Setup

In [1]:
# SWA weights
!/home/jupyter/.local/bin/gdown --id 1-0mtggsTMLDmbT7cmxdIBH25eAieTyGb
!/home/jupyter/.local/bin/gdown --id 1--EvBdSAnNaBtKozkMuja--lq8erN0uC

# MA weights
!/home/jupyter/.local/bin/gdown --id 1M0JwphtV6W1NM6V2Qv_VMTdjWdeAz6Sx
!/home/jupyter/.local/bin/gdown --id 1pW2hx6fkQGH8fXFfh8Fn8xPADuYBy5mG

Downloading...
From: https://drive.google.com/uc?id=1-0mtggsTMLDmbT7cmxdIBH25eAieTyGb
To: /home/jupyter/teacher_model_swa.h5
94.6MB [00:01, 83.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--EvBdSAnNaBtKozkMuja--lq8erN0uC
To: /home/jupyter/student_noisy_swa.h5
94.6MB [00:01, 85.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1M0JwphtV6W1NM6V2Qv_VMTdjWdeAz6Sx
To: /home/jupyter/teacher_model_ma.h5
94.6MB [00:01, 71.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pW2hx6fkQGH8fXFfh8Fn8xPADuYBy5mG
To: /home/jupyter/student_noisy_ma.h5
94.6MB [00:01, 63.9MB/s]


In [2]:
from tensorflow.keras import layers
import tensorflow as tf

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tqdm import tqdm
import numpy as np

## Define Hyperparameters

In [3]:
AUTO = tf.data.AUTOTUNE
DATASET_NAME = "cifar10_corrupted"
BATCH_SIZE = 64
IMAGE_SIZE = 72

In [4]:
VERSIONS = [
    "brightness_5",
    "contrast_5",
    "defocus_blur_5",
    "elastic_5",
    "fog_5",
    "frost_5",
    "frosted_glass_blur_5",
    "gaussian_blur_5",
    "gaussian_noise_5",
    "impulse_noise_5",
    "jpeg_compression_5",
    "motion_blur_5",
    "pixelate_5",
    "saturate_5",
    "shot_noise_5",
    "snow_5",
    "spatter_5",
    "speckle_noise_5",
    "zoom_blur_5"
]

print(f"Total sub-versions of the CIFAR10-C dataset: {len(VERSIONS)}")

Total sub-versions of the CIFAR10-C dataset: 19


## Utilities

In [5]:
def prepare_dataset(ds):
    ds = (ds
          .batch(BATCH_SIZE)
          .map(lambda x, y: (tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE)), y), 
               num_parallel_calls=AUTO)
          .prefetch(AUTO)
    )
    return ds

In [6]:
def get_training_model(num_classes=10):
    resnet50_v2 = tf.keras.applications.ResNet50V2(
        weights=None,
        include_top=False,
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    )
    model = tf.keras.Sequential(
        [
            layers.Input((IMAGE_SIZE, IMAGE_SIZE, 3)),
            layers.experimental.preprocessing.Rescaling(scale=1.0 / 127.5, offset=-1),
            resnet50_v2,
            layers.GlobalAveragePooling2D(),
            layers.Dense(num_classes)
        ]
    )
    return model

In [7]:
def evaluate_model(model):
    acc_dict = {}
    for version in tqdm(VERSIONS):
        print(f"Processing {version}")
        dataset_fullname = DATASET_NAME + "/" + version
        loaded_ds = tfds.load(
            dataset_fullname,
            split="test",
            as_supervised=True
        )
        loaded_ds = prepare_dataset(loaded_ds)
        _, acc = model.evaluate(loaded_ds, verbose=0)
        print(f"Test accuracy on {version}: {acc*100}%")
        acc_dict[version] = acc*100
        
    return acc_dict, np.mean(list(acc_dict.values()))

## Evaluation

### SWA

In [8]:
# Evaluate teacher model trained with SWA
teacher_model_swa = get_training_model()
teacher_model_swa.load_weights("teacher_model_swa.h5")
teacher_model_swa.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model_swa)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:19<05:43, 19.09s/it]

Test accuracy on brightness_5: 76.05999708175659%
Processing contrast_5


 11%|█         | 2/19 [00:21<02:36,  9.18s/it]

...

Test accuracy on spatter_5: 64.16000127792358%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:58<00:02,  2.28s/it]

Test accuracy on speckle_noise_5: 45.62999904155731%
Processing zoom_blur_5


100%|██████████| 19/19 [01:00<00:00,  3.18s/it]

Test accuracy on zoom_blur_5: 77.49000191688538%
Mean Top-1 Accuracy: 59.695262579541456%


In [9]:
# Evaluate the corresponding student model
student_noisy_swa = get_training_model()
student_noisy_swa.load_weights("student_noisy_swa.h5")
student_noisy_swa.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy_swa)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<01:00,  3.35s/it]

Test accuracy on brightness_5: 82.91000127792358%
Processing contrast_5


 11%|█         | 2/19 [00:05<00:44,  2.64s/it]

...

Test accuracy on spatter_5: 70.31999826431274%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:39<00:02,  2.14s/it]

Test accuracy on speckle_noise_5: 53.46999764442444%
Processing zoom_blur_5


100%|██████████| 19/19 [00:42<00:00,  2.21s/it]

Test accuracy on zoom_blur_5: 79.67000007629395%
Mean Top-1 Accuracy: 62.80894765728399%


### MA

In [10]:
# Evaluate teacher model trained with MA
teacher_model_ma = get_training_model()
teacher_model_ma.load_weights("teacher_model_ma.h5")
teacher_model_ma.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(teacher_model_ma)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<01:00,  3.37s/it]

Test accuracy on brightness_5: 73.14000129699707%
Processing contrast_5


 11%|█         | 2/19 [00:05<00:45,  2.65s/it]

...

Test accuracy on spatter_5: 63.8700008392334%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:40<00:02,  2.18s/it]

Test accuracy on speckle_noise_5: 49.75000023841858%
Processing zoom_blur_5


100%|██████████| 19/19 [00:42<00:00,  2.24s/it]

Test accuracy on zoom_blur_5: 78.32000255584717%
Mean Top-1 Accuracy: 59.89315792133934%


In [11]:
# Evaluate the corresponding student model
student_noisy_ma = get_training_model()
student_noisy_ma.load_weights("student_noisy_ma.h5")
student_noisy_ma.compile(loss="sparse_categorical_crossentropy",
                         metrics=["accuracy"])
acc_dict, mean_top_1 = evaluate_model(student_noisy_ma)
print(f"Mean Top-1 Accuracy: {mean_top_1}%")

  0%|          | 0/19 [00:00<?, ?it/s]

Processing brightness_5


  5%|▌         | 1/19 [00:03<00:59,  3.32s/it]

Test accuracy on brightness_5: 81.80999755859375%
Processing contrast_5


 11%|█         | 2/19 [00:05<00:44,  2.63s/it]

...

Test accuracy on spatter_5: 70.24000287055969%
Processing speckle_noise_5


 95%|█████████▍| 18/19 [00:40<00:02,  2.17s/it]

Test accuracy on speckle_noise_5: 52.27000117301941%
Processing zoom_blur_5


100%|██████████| 19/19 [00:42<00:00,  2.22s/it]

Test accuracy on zoom_blur_5: 77.46000289916992%
Mean Top-1 Accuracy: 61.863684340527186%
